In [2]:
import pandas as pd
import re
import pkg_resources
import requests
from symspellpy.symspellpy import SymSpell, Verbosity  # its a spell check library
max_edit_distance_dictionary = 2
prefix_length = 7
# create object
sym_spell = SymSpell(max_edit_distance_dictionary, prefix_length)
# load dictionary
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
bigram_path = pkg_resources.resource_filename("symspellpy", "frequency_bigramdictionary_en_243_342.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
if not sym_spell.load_dictionary(dictionary_path, term_index=0,count_index=1):
        print("Dictionary file not found")
if not sym_spell.load_bigram_dictionary(bigram_path, term_index=0,count_index=2):
        print("Bigram dictionary file not found")

In [19]:
input_term = 'disc.'
# max edit distance per lookup (per single word, not per whole input string)
max_edit_distance_lookup = 2
suggestions = sym_spell.lookup_compound(input_term,max_edit_distance_lookup)
# display suggestion term, edit distance, and term frequency
for suggestion in suggestions:
    print("{}, {}, {}".format(suggestion.term, suggestion.distance,suggestion.count))

disc, 1, 34140094


In [51]:
d={"Seller State":"","Seller ID":"","Seller Name":"","Seller Address":"","Seller GSTIN Number":"","Country of Origin":"","Currency":"","Description":""}
def seller_state(df,d):
    df=df.drop_duplicates()
    for i in df.index:
        description=df['key'][i].lower()
        if(description.find("place of supply")!=-1):
            print("Seller state:",df['value'][i])
            d.update({"Seller State":df['value'][i]})
        elif(isinstance(df['value'][i],str)):
            description=df['value'][i].lower()
            if(description.find("place of supply")!=-1):
                d.update({"Seller State":description.split("place of supply",1)[1]})
def seller_id(rt):
    for i in rt:
        i=i.lower()
        if('cin' in i):
            found=i.find(":")
            if(found!=-1):
                d.update({"Seller ID":i.split(":")[1].replace("\n",'').strip(' ').strip("-")})
            else:
                found=i.find(".")
                if(found!=-1):
                    d.update({"Seller ID":i.split(".")[1].replace("\n",'').strip(' ').strip("-")})
            break
def seller_gst(rt):
    nxt_line=0
    for i in rt:
        i=i.lower()
        if(nxt_line):
            Gst_no=i.strip("\n").strip(" ")
            if(Gst_no.isalnum() and len(Gst_no)==15):
                d.update({"Seller GSTIN Number":Gst_no.replace("-","")})
            break
        if('gst' in i):
            found=i.find(":")
            if(found!=-1):
                Gst_no=i.split(":")[1].strip("\n").strip(" ")
                if(Gst_no.isalnum() and len(Gst_no)==15):
                    d.update({"Seller GSTIN Number":Gst_no.replace("-","")})
                break
            else:
                found=i.find(".")
                if(found!=-1):
                    Gst_no=i.split(".")[1].strip("\n").strip(" ")
                    if(Gst_no.isalnum() and len(Gst_no)==15):
                        d.update({"Seller GSTIN Number":Gst_no.replace("-","")})
                    break
                else:
                    nxt_line=1
def currency(rt):
    for i in rt:
        i=i.lower()
        if("currency" in i):
            found=i.find(":")
            if(found!=-1):
                d.update({"Currency":i.split(":")[1].strip(" ").strip("-").replace("\n","")})
                break
        if(("inr" in i) or ("rupees" in i)):
            d.update({"Currency":"INR"})
            break
def country_of_origin(rt):
    f=1
    dict_of_currencies={'INR':'INDIA'}
    for i in rt:
        i=i.lower()
        if("country of origin" in i):
            found=i.find(":")
            if(found!=-1):
                d.update({"Country of Origin":i.split(":")[1].strip(" ").strip("-").replace("\n","").replace("/","")})
                f=0
                break
    if(f):
        d.update({"Country of Origin":dict_of_currencies[d['Currency']]})
def address(rt):
    for i in rt:        
        if(i.count(",")>1):
            d.update({"Seller Address":i.strip("\n")})
            break
with open('keyValues.csv') as f:
    df=pd.read_csv(f)
    seller_state(df,d)
with open('rawText.txt') as f:
    l=f.readlines()
    text=" ".join(i for i in l)
    seller_id(l)
    seller_gst(l)
    currency(l)
    country_of_origin(l)
    address(l)

In [30]:
def table_content():
    df=pd.read_csv('table-1.csv')
    df.dropna(axis=0,thresh=5,inplace=True)
    l=[i.lower().strip(' ') for i in df.columns]
    d=dict()
    # max edit distance per lookup (per single word, not per whole input string)
    max_edit_distance_lookup = 2
    cols=['Product code','HSN','Title','Quantity','Unit Price']
    for i in l:
        input_term = i
        suggestions = sym_spell.lookup_compound(input_term,max_edit_distance_lookup)
        for suggestion in suggestions:
            j=suggestion.term
        if(('material' in i) or ('product code') in i):
            d.update({df.columns[l.index(i)]:cols[0]})
        elif(('hsn' in i) or ('code'==i)):
            d.update({df.columns[l.index(i)]:cols[1]})
        elif(('description' in i) or('description' in j) or('daecripiior' in i) or('dascettion' in i)):
            d.update({df.columns[l.index(i)]:cols[2]})
        elif(('quantity' in i) or ((('qty' in i)or ('oty' in i)))):
            d.update({df.columns[l.index(i)]:cols[3]})
        elif(('Unit price' in i) or ('rate'==i) or("per unit" in i)):
            d.update({df.columns[l.index(i)]:cols[4]})
    df=df.rename(columns=d)
    df.drop(df.columns.difference(cols),axis=1,inplace=True)
    df.to_csv("Product_info.csv")
    print(df)
table_content()

            Description of Goods     HSN/SAC GST Rate Quantity    Rate  per   \
0  DUMMY PRODUCT FOR EVALUATION   94054090.0      12%  74 PCS   247.32  PCS    
1  DUMMY PRODUCT FOR EVALUATION   94054090.0     129%  50 PCS   494.64  PCS    
2  DUMMY PRODUCT FOR EVALUATION   94054090.0   12% %   50 PCS   539.29  PCS    
3  DUMMY PRODUCT FOR EVALUATION   94054090.0      12%  74 PCS   269.64  PCS    
4  DUMMY PRODUCT FOR EVALUATION   94054090.0      12%  50 PCS   989.29  PCS    

   Disc.      Amount   
0     NaN  18.301.68   
1     NaN  24.732.00   
2     NaN  26.964.50   
3     NaN  19.953.36   
4     NaN  49.464.50   
                           Title         HSN Quantity  Unit Price
0  DUMMY PRODUCT FOR EVALUATION   94054090.0  74 PCS       247.32
1  DUMMY PRODUCT FOR EVALUATION   94054090.0  50 PCS       494.64
2  DUMMY PRODUCT FOR EVALUATION   94054090.0  50 PCS       539.29
3  DUMMY PRODUCT FOR EVALUATION   94054090.0  74 PCS       269.64
4  DUMMY PRODUCT FOR EVALUATION   94054090.0 

In [3]:
def get_headers(filename):
    with open(filename,'r') as f:
        t=f.readlines()
        l=[(i.strip("\n")).lower() for i in t]
        index=-1
        if("si" in l):
            index=l.index("si")
        elif("sl.no" in l):
            index=l.index("sl.no")
        elif("sl no" in l):
            index=l.index("sl no")
        elif("sl" in l):
            index=l.index("sl no")
        if(index!=-1):
            i=index+1
            numindex=-1
            numcheck=1
            while(i<len(l) and numcheck):
                numindex=i
                if(l[i].isdigit()):
                    numcheck=0
                i+=1
            if(numindex!=-1):
                no_of_columns=numindex-index-2
                df=pd.read_csv("table-11.csv")
                print(df)
                last_column=df.columns[len(df.columns)-2]
                first_column=df.columns[0]
                li=l.index(last_column.lower().strip())
                if(li<numindex):
                    cols=t[li-no_of_columns+1:li+1]
                    cols.append("")
                    df.columns=cols
                    print(df)
get_headers("rawText.txt")

                                      Unnamed: 0  Unnamed: 1 Unnamed: 2  \
0                  DUMMY PRODUCT FOR EVALUATION   94054090.0       12%    
1                  DUMMY PRODUCT FOR EVALUATION   94054090.0     129 %    
2                  DUMMY PRODUCT FOR EVALUATION   94054090.0     12% %    
3                  DUMMY PRODUCT FOR EVALUATION   94054090.0      12 %    
4                  DUMMY PRODUCT FOR EVALUATION   94054090.0       12%    
5  Output IGST@12% Rounded Off (Short & Excess)          NaN        NaN   
6                                         Total          NaN        NaN   

  Unnamed: 3  Unnamed: 4  per   Disc.                       Amount   \
0    74 PCS       247.32  PCS      NaN                   18.301.68    
1    50 PCS       494.64  PCS      NaN                   24.732.00    
2    50 PCS       539.29  PCS      NaN                   26.964.50    
3    74 PCS       269.64  PCS      NaN                   19.953.36    
4    50 PCS       989.29  PCS      NaN      